In [1]:
import boto3

In [2]:
%%writefile Dockerfile

FROM python:3.9-slim-buster

COPY requirements.txt /opt/ml/requirements.txt
COPY code/preprocess.py /opt/ml/code/preprocess.py
COPY code/train.py /opt/ml/code/train.py

RUN pip3 install -r /opt/ml/requirements.txt

ENV SAGEMAKER_PROGRAM preprocess.py
ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3", "/opt/ml/code/train.py"]

Overwriting Dockerfile


In [3]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'demo_prophet'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [4]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository .

Sending build context to Docker daemon  175.1kB
Step 1/9 : FROM python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/9 : COPY requirements.txt /opt/ml/requirements.txt
 ---> Using cache
 ---> 7edd8862d744
Step 3/9 : COPY code/preprocess.py /opt/ml/code/preprocess.py
 ---> c28480fa2c88
Step 4/9 : COPY code/train.py /opt/ml/code/train.py
 ---> 99906b8f8449
Step 5/9 : RUN pip3 install -r /opt/ml/requirements.txt
 ---> Running in ab03f5f26008
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 33.6 MB/s eta 0:

In [5]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [6]:
!aws ecr create-repository --repository-name $ecr_repository


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'demo_prophet' already exists in the registry with id '654654222480'


In [7]:
# aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
# if [ $? -ne 0 ]
# then
# aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
# fi

In [8]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [9]:
!docker push $processing_repository_uri

The push refers to repository [654654222480.dkr.ecr.us-east-1.amazonaws.com/demo_prophet]

80f427d4: Preparing 
b032df9a: Preparing 
640c4bb4: Preparing 
dcbaed91: Preparing 
a27560c1: Preparing 
037e08b3: Preparing 
eede8d6e: Preparing 
55769c5e: Preparing 
80f427d4: Pushed   657.9MB/627.5MBAPushing  69.86MB/627.5MBlatest: digest: sha256:387eec54c7f79849089acae0b115a0cba4a8e6b93e356deb6720cc41e651bfdd size: 2204
